# Code for Thaloz Test - TASKS :

1- Reads and preprocesses customer service conversations from a CSV file.  (check)  
2- Generates embeddings for customer messages using sentence-transformers.  (check)  
3- Stores embeddings in a vector database (FAISS) for fast similarity search.  (check)  
4- Implements a function to retrieve the top 3 most relevant responses based on a user query.  (Check)  

In [1]:
import pandas as pd

df = pd.read_csv("customer_service_conversations.csv", index_col="ID")
df

,Customer Message,Agent Response
ID,,
1,My order hasn’t arrived yet. Can you check?,I’m sorry for the delay! Can you provide your ...
2,I need to reset my password but the link is br...,I understand. Let me generate a new reset link...
3,Do you have a refund policy?,Yes! We offer a 30-day money-back guarantee. W...
4,The product I received is defective.,I'm sorry to hear that. We can arrange a repla...
5,How can I contact support?,You can reach us via email at support@example....


In [2]:
from sentence_transformers import SentenceTransformer

# Load the pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')
df['embeddings'] = df['Customer Message'].apply(lambda x: model.encode(x))


In [3]:
import faiss
import numpy as np

# Convert embeddings to a numpy array
embedding_matrix = np.array(df['embeddings'].tolist()).astype('float32')

# Create a FAISS index
index = faiss.IndexFlatL2(embedding_matrix.shape[1])  # Using L2 distance
index.add(embedding_matrix)  # Add embeddings to the index

# Function to retrieve top 3 most similar responses
def retrieve_top_responses(query, k=3):
    query_embedding = model.encode(query).reshape(1, -1).astype('float32')
    distances, indices = index.search(query_embedding, k)  # Search for the top k
    return df.iloc[indices[0]]  # Return the top k responses


In [4]:
def get_relevant_responses():
    for message in df['Customer Message']:
        top_responses = retrieve_top_responses(message)
        print(f"Customer Message: {message}")
        print("Top 3 Agent Responses:")
        for response in top_responses['Agent Response']:
            print(f"- {response}")
        print("\n")


In [5]:
from sklearn.metrics.pairwise import cosine_similarity

def get_most_relevant_response(query):
    # Get the top response embeddings
    top_responses = retrieve_top_responses(query)
    
    # Calculate cosine similarity between the query and top responses
    query_embedding = model.encode(query).reshape(1, -1).astype('float32')
    response_embeddings = np.array(top_responses['embeddings'].tolist()).astype('float32')
    
    similarities = cosine_similarity(query_embedding, response_embeddings).flatten()
    
    # Get the index of the most relevant response
    most_relevant_index = np.argmax(similarities)
    
    return top_responses.iloc[most_relevant_index]

# Example usage
for message in df['Customer Message']:
    top_responses = retrieve_top_responses(message)
    print(f"Customer Message: {message}")
    print("Top 3 Agent Responses:")
    for response in top_responses['Agent Response']:
        print(f"- {response}")
    print("\n")


Customer Message: My order hasn’t arrived yet. Can you check?
Top 3 Agent Responses:
- I’m sorry for the delay! Can you provide your order number?
- I'm sorry to hear that. We can arrange a replacement or refund. What would you prefer?
- Yes! We offer a 30-day money-back guarantee. Would you like me to process a refund?


Customer Message: I need to reset my password but the link is broken.
Top 3 Agent Responses:
- I understand. Let me generate a new reset link for you.
- You can reach us via email at support@example.com or call our helpline.
- I'm sorry to hear that. We can arrange a replacement or refund. What would you prefer?


Customer Message: Do you have a refund policy?
Top 3 Agent Responses:
- Yes! We offer a 30-day money-back guarantee. Would you like me to process a refund?
- You can reach us via email at support@example.com or call our helpline.
- I’m sorry for the delay! Can you provide your order number?


Customer Message: The product I received is defective.
Top 3 Agent